# 모델에 들어가지 않는 데이터

In [1]:
import sys
print(sys.path)

['/home/ubuntu/predict', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/ubuntu/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


영화 상세정보 api에 크롤링 2개 merge한 데이터 프레임명이 `df_info` 일때

## 1) 장르, 등급, 감독, 국적, 배급사 값 추출

In [2]:
import pandas as pd
import numpy as np
import boto3
import datetime
from io import StringIO

In [3]:
def download_file_s3(bucket):
    client = boto3.client('s3')
    bucket = 'movietomodel'
    date = str(datetime.datetime.now() - datetime.timedelta(1))[:10].replace('-', '')
    file = client.get_object(Bucket = bucket, Key='yuha/yuha-'+date+'.csv')
    body = file['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    return df

In [4]:
df_info = download_file_s3('modeltomovie')
df_info = df_info.drop('Unnamed: 0', axis = 1)

## 3) 배급사/감독 흥행지수 조인

In [7]:
distri_index = pd.read_csv('~/predict/company.csv') #저장된 경로로 바꾸기
direc_index = pd.read_csv('~/predict/director.csv')

mean_distri = distri_index.mean()[0]
mean_direc = direc_index.mean()[0]

/tmp/ipykernel_1326/221427427.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_distri = distri_index.mean()[0]
/tmp/ipykernel_1326/221427427.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_direc = direc_index.mean()[0]


In [8]:
def add_index(data):
  data = data.merge(distri_index[['배급사', '흥행지수1']],on='배급사', how='left')
  data = data.merge(direc_index[['감독', '흥행지수1']],on='감독', how='left')
  data.rename(columns={'흥행지수1_x':'배급사 흥행지수', '흥행지수1_y':'감독 흥행지수'}, inplace=True)
  data['배급사 흥행지수'] = data['배급사 흥행지수'].fillna(mean_distri)
  data['감독 흥행지수'] = data['감독 흥행지수'].fillna(mean_direc)
  return data

In [9]:
data = add_index(df_info)

# 모델 전처리~ 로드

## 1) 모델에 들어갈 데이터 전처리

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import pickle
import joblib

In [11]:
# 모델에 넣을 용도 ; 필요한 컬럼만 가져오기 (맨마지막 실행)
def drop_columns(df):
  test = df[['movieCd','openDt','감독 흥행지수', '뉴스 언급 건수','배급사 흥행지수', '장르', '국적', '등급', '평점']]
  test['movieCd'] = df['movieCd'].astype(int)
  test['openDt'] = df['movieCd'].astype(int)
  return test

In [12]:
test = drop_columns(data)

/tmp/ipykernel_1326/2411545346.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['movieCd'] = df['movieCd'].astype(int)
/tmp/ipykernel_1326/2411545346.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['openDt'] = df['movieCd'].astype(int)


## 2) 라벨 인코딩

In [13]:
def change_genre(x):
  genre_list = ['드라마', '판타지', 'SF', '액션', '애니메이션', '다큐멘터리', '사극', '코미디', '어드벤처','스릴러', '범죄', '공포(호러)', '멜로/로맨스', '뮤지컬', '미스터리', '공연', '가족', '전쟁','기타', '서부극(웨스턴)']
  return genre_list.index(x)

def change_nation(x):
  if x == '한국':
    return 2
  elif x == '미국':
    return 1
  else:       # '한국미국제외'
    return 0 

def change_grade(x): 
  if x == '전체관람가':
    return 3
  elif x == '12세이상관람가':
    return 2
  elif x == '15세이상관람가':
    return 1
  else:
    return 0 

In [14]:
test['장르'] = test['장르'].map(change_genre) 
test['국적'] = test['국적'].map(change_nation) 
test['등급'] = test['등급'].map(change_grade) 

/tmp/ipykernel_1326/2282469254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['장르'] = test['장르'].map(change_genre)
/tmp/ipykernel_1326/2282469254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['국적'] = test['국적'].map(change_nation)
/tmp/ipykernel_1326/2282469254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

## 3) 모델 load

In [15]:
final_model = joblib.load('modelpredict_final_model_label.pkl')  # path 추가 필요!

/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## 4) 예측

In [16]:
test_features = test.drop(["movieCd"], axis = 1)

test_predict = final_model.predict(test_features)

In [17]:
test['predict'] = test_predict

/tmp/ipykernel_1326/4107097388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predict'] = test_predict


In [18]:
data['predict'] = test['predict']
data

,movieCd,movieNm,movieNmEn,movieNmOg,showTm,prdtYear,openDt,prdtStatNm,typeNm,nations,...,뉴스 언급 건수,평점,장르,등급,감독,국적,배급사,배급사 흥행지수,감독 흥행지수,predict
0,20194376,탑건: 매버릭,Top Gun: Maverick,NaN,130,2021,20220622,개봉,장편,[{'nationNm': '미국'}],...,6,9.77,액션,12세이상관람가,조셉 코신스키,미국,롯데컬처웍스(주)롯데엔터테인먼트,0.484757,0.043822,1.758303e+06
1,20224882,마녀(魔女) Part2. The Other One,The Witch: Part 2. The Other One,NaN,137,2021,20220615,개봉,장편,[{'nationNm': '한국'}],...,0,6.66,액션,15세이상관람가,박훈정,한국,(주)넥스트엔터테인먼트월드(NEW),0.435840,0.155757,4.070401e+06
2,20204548,범죄도시 2,The Roundup,NaN,105,2022,20220518,개봉,장편,[{'nationNm': '한국'}],...,108,9.46,범죄,15세이상관람가,이상용,한국,주식회사 에이비오엔터테인먼트,0.257650,0.029272,2.570272e+06
3,20206257,브로커,Broker,NaN,129,2022,20220608,개봉,장편,[{'nationNm': '한국'}],...,101,5.51,드라마,12세이상관람가,고레에다 히로카즈,한국,(주)씨제이이엔엠,0.602172,0.004183,4.775629e+05
4,20223839,버즈 라이트이어,Lightyear,NaN,104,2022,20220615,개봉,장편,[{'nationNm': '미국'}],...,3,7.73,애니메이션,전체관람가,앵거스 맥클레인,미국,월트디즈니컴퍼니코리아 유한책임회사,0.878787,0.029272,2.095207e+06
5,20209654,헤어질 결심,Decision To Leave,NaN,138,2022,20220629,개봉예정,장편,[{'nationNm': '한국'}],...,97,0.00,미스터리,15세이상관람가,박찬욱,한국,(주)씨제이이엔엠,0.602172,0.131670,5.923861e+06
6,20206061,쥬라기 월드: 도미니언,Jurassic World: Dominion,NaN,146,2022,20220601,개봉,장편,[{'nationNm': '미국'}],...,6,6.23,액션,12세이상관람가,콜린 트레보로우,미국,유니버설픽쳐스인터내셔널 코리아(유),0.233966,0.314524,4.512785e+06
7,20200155,모어,I am More,NaN,81,2021,20220623,개봉,장편,[{'nationNm': '한국'}],...,11,9.17,다큐멘터리,15세이상관람가,이일하,한국,(주)엣나인필름,0.014354,0.029272,9.533089e+04
8,20225114,극장판 윌벤져스 : 수상한 캠핑 대소동,NaN,NaN,79,2022,20220623,개봉,장편,[{'nationNm': '한국'}],...,0,9.90,애니메이션,전체관람가,신창환,한국,(주)대교 미디어콘텐츠사업본부,0.003219,0.029272,3.001278e+05
9,20224757,룸 쉐어링,My Perfect Roommate,NaN,92,2021,20220622,개봉,장편,[{'nationNm': '한국'}],...,2,9.28,드라마,전체관람가,이순성,한국,주식회사 엔픽플,0.076040,0.029272,8.595284e+05


# 3.일별 박스오피스 데이터에서 대시보드에 들어가야 하는 데이터

In [19]:
import json
import pandas as pd
import numpy as np
import requests
import re

def date_format(date) : 
    date = str(date)[:10]
    date = date.replace('-','')
    return date

def getMovie10() :
    date = str(datetime.datetime.now() - datetime.timedelta(1))[:10].replace('-', '')
    url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5d450765c401dac01475905006e44d99&targetDt='+date
    res = requests.get(url)
    text= res.text
    movie_raw = json.loads(text)
    df = pd.DataFrame()
    l1 = []; l2 = []; l3 = []; l4 = []; l5 =[]; l6 = []; l7 = [];
    l8 = []; l9 = []; l10 = []; l11 = []; l12 = []; l13 = []; l14 = []; l15 = [];
    
    for movie_info in movie_raw['boxOfficeResult']['dailyBoxOfficeList']:
        l1.append(movie_info['movieNm'])
        l2.append(movie_info['openDt'])
        l3.append(movie_info['movieCd']) 
        l4.append(movie_info['rank'])
        l5.append(movie_info['rankInten'])
        l6.append(movie_info['rankOldAndNew'])
        l7.append(movie_info['salesAmt'])
        l8.append(movie_info['salesShare'])
        l9.append(movie_info['salesChange'])
        l10.append(movie_info['salesAcc'])
        l11.append(movie_info['audiCnt'])
        l12.append(movie_info['audiInten'])
        l13.append(movie_info['audiChange'])
        l14.append(movie_info['audiAcc'])
        l15.append(movie_info['audiChange'])
    
    df['movieNm'] = l1
    df['openDt'] = [int(re.sub('-','',i)) for i in l2]
    df['movieCd'] = [int(i) for i in l3 ]  
    df['rank'] = [int(i) for i in l4]   
    df['rankInten'] = [int(i) for i in l5]   
    df['rankOldAndNew'] = l6   
    df['salesAmt'] = [int(i) for i in l7]   
    df['salesShare'] = [float(i) for i in l8]   
    df['salesChange'] = [float(i) for i in l9]   
    df['salesAcc'] = [int(i) for i in l10]   
    df['audiCnt'] = [int(i) for i in l11]   
    df['audiInten'] = [int(i) for i in l12]   
    df['audiChange'] = [float(i) for i in l13]     
    df['audiAcc'] = [float(i) for i in l14]     
    df['audiChange'] = [float(i) for i in l15]      
    return df

In [20]:
data2 = getMovie10()

In [21]:
data2['movieCd'] = data2['movieCd'].astype(int)
data_all = pd.merge(data, data2, how='left', on = ['movieCd', 'openDt','movieNm'])

In [22]:
data_last = data_all.drop(['movieNmEn', 'movieNmOg', 'prdtYear', 'nations', 'genres', 'directors', 'actors', 'showTypes', 'companys', 'audits', 'staffs', 'openDt_c', 'startDt', 'endDt'], axis=1)

In [23]:
data_last

,movieCd,movieNm,showTm,openDt,prdtStatNm,typeNm,뉴스 언급 건수,평점,장르,등급,...,rankInten,rankOldAndNew,salesAmt,salesShare,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc
0,20194376,탑건: 매버릭,130,20220622,개봉,장편,6,9.77,액션,12세이상관람가,...,0,OLD,1546652190,50.5,-21.7,4263654550,146128,-42190,-22.4,399150.0
1,20224882,마녀(魔女) Part2. The Other One,137,20220615,개봉,장편,0,6.66,액션,15세이상관람가,...,0,OLD,703482280,23.0,-15.9,18930191930,69692,-13005,-15.7,1839179.0
2,20204548,범죄도시 2,105,20220518,개봉,장편,108,9.46,범죄,15세이상관람가,...,0,OLD,537730050,17.6,-6.9,121199598990,53998,-3876,-6.7,11725561.0
3,20206257,브로커,129,20220608,개봉,장편,101,5.51,드라마,12세이상관람가,...,0,OLD,88606770,2.9,-8.6,11705090130,9141,-852,-8.5,1169802.0
4,20223839,버즈 라이트이어,104,20220615,개봉,장편,3,7.73,애니메이션,전체관람가,...,0,OLD,32607990,1.1,-6.1,2526609680,3818,-134,-3.4,237099.0
5,20209654,헤어질 결심,138,20220629,개봉예정,장편,97,0.00,미스터리,15세이상관람가,...,5,OLD,32616000,1.1,487.5,89542000,3624,3008,488.3,9946.0
6,20206061,쥬라기 월드: 도미니언,146,20220601,개봉,장편,6,6.23,액션,12세이상관람가,...,-1,OLD,20987860,0.7,-1.1,28798469770,2208,-63,-2.8,2790751.0
7,20200155,모어,81,20220623,개봉,장편,11,9.17,다큐멘터리,15세이상관람가,...,12,OLD,10107700,0.3,461.5,21548700,1373,1073,357.7,2805.0
8,20225114,극장판 윌벤져스 : 수상한 캠핑 대소동,79,20220623,개봉,장편,0,9.90,애니메이션,전체관람가,...,0,NEW,9964400,0.3,100.0,12432400,1339,1339,100.0,1663.0
9,20224757,룸 쉐어링,92,20220622,개봉,장편,2,9.28,드라마,전체관람가,...,-3,OLD,9572700,0.3,-27.9,43015260,1152,-434,-27.4,5089.0


In [24]:
data_last = data_last.to_csv()

In [25]:
from datetime import datetime, timedelta

In [26]:
def upload_file_s3(bucket,filename, content):
    client = boto3.client('s3')
    try :
        client.put_object(Bucket=bucket, Key=filename, Body=content)
        return True
    except :
        return False

bucket = 'modeltoelastic'
date = str(datetime.now() - timedelta(1))[:10].replace('-', '')
file_name = 'yuha-'+date
upload_file_s3(bucket,'yuha/' + file_name + '.csv', data_last)

True